# New York Air Quality Data Mining Project
Zeb Moffat  
Yida Fang

## Data Cleaning

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
air_quality = pd.read_csv('./data/New_York_Air_Quality.csv')

#remove completely blank Message column
air_quality.drop(columns=["Message"], inplace=True)

   Unique ID  Indicator ID                    Name Measure Measure Info  \
0     336867           375  Nitrogen dioxide (NO2)    Mean          ppb   
1     336741           375  Nitrogen dioxide (NO2)    Mean          ppb   
2     550157           375  Nitrogen dioxide (NO2)    Mean          ppb   
3     412802           375  Nitrogen dioxide (NO2)    Mean          ppb   
4     412803           375  Nitrogen dioxide (NO2)    Mean          ppb   

  Geo Type Name  Geo Join ID                     Geo Place Name  \
0            CD          407      Flushing and Whitestone (CD7)   
1            CD          107              Upper West Side (CD7)   
2            CD          414  Rockaway and Broad Channel (CD14)   
3            CD          407      Flushing and Whitestone (CD7)   
4            CD          407      Flushing and Whitestone (CD7)   

           Time Period  Start_Date  Data Value  
0       Winter 2014-15  12/01/2014       23.97  
1       Winter 2014-15  12/01/2014       27.42  

## Association

### Method 1 (Rename)

### Method 2 (Rename)

## Classification

### Method 1 (Rename)

### Method 2 (Rename)

## Clustering

### Method 1 (Rename)

### Method 2 (Rename)